<a href="https://colab.research.google.com/github/Olanle/Project-006-House-price-predictor-full-pipeline-with-deployed-notebook/blob/main/006.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#Load & Inspect House Price Dataset

import pandas as pd
import numpy as np

df = pd.read_csv("/content/Housing.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/content/Housing.csv'

In [2]:
print("Rows, Columns:", df.shape)

print("\n--- Head ---")
print(df.head())


print("\n--- Info ---")
df.info()

print("\n--- Numeric Describe ---")
print(df.describe().T)

print("\n--- Missing values ---")
missing = df.isnull().sum().sort_values(ascending=False)
print(missing[missing > 0])

NameError: name 'df' is not defined

In [ ]:
#Check target distribution / summary
if 'price' in df.columns:
    print("\n--- Target (SalePrice) stats ---")
    print(df['price'].describe())

    # quick skew check
    print("Skew:", df['price'].skew())
else:
    print("\nWARNING: 'price' not found. Set the correct target column name.")

#Show categorical columns and a sample of unique values (first 10)
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
print(f"\nCategorical columns ({len(cat_cols)}): {cat_cols}")

print("\n")
for c in cat_cols:
    print(f" - {c}: {df[c].nunique()} uniques; sample -> {df[c].dropna().unique()[:10]}")

In [ ]:
#Define Features, Target & Split Data

from sklearn.model_selection import train_test_split

target = 'price'

#Drop identifier or irrelevant columns
drop_cols = ['Id'] if 'Id' in df.columns else []

#Define features (everything except target + dropped columns)
X = df.drop(columns=[target] + drop_cols)
y = df[target]

#Train-test split (80-20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training data: {X_train.shape}")
print(f"Testing data:  {X_test.shape}")


In [ ]:
#Preprocessing Pipeline

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

# Separate feature types
num_features = X_train.select_dtypes(include=['int64', 'float64']).columns
cat_features = X_train.select_dtypes(include=['object']).columns

# Numeric pipeline
num_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Categorical pipeline
cat_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Combine both
preprocessor = ColumnTransformer(transformers=[
    ('num', num_pipeline, num_features),
    ('cat', cat_pipeline, cat_features)
])

print("✅ Preprocessing pipeline created successfully!")
print(f"Numerical features: {len(num_features)}")
print(f"Categorical features: {len(cat_features)}")
